In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import ipywidgets as widgets
import pandas as pd

In [4]:
# load environments variables from .env file
load_dotenv()

True

In [5]:
iteratec_ml_server_base_url=os.environ.get("ITERATEC_ML_SERVER_URL")
iteratec_ml_server_api_key=os.environ.get("ITERATEC_ML_SERVER_API_KEY")
print(iteratec_ml_server_base_url)

http://mlserver.iteratec.de:8000/v1


In [5]:
client = OpenAI(
    base_url=iteratec_ml_server_base_url,
    api_key=iteratec_ml_server_api_key
)

In [7]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What are some good projects to get into robotics?",
        }
    ],
    model="casperhansen/mixtral-instruct-awq"
)
print(chat_completion.choices[0].message.content)



 There are many great projects to get into robotics, depending on your skill level, interests, and resources. Here are some project ideas for various levels:

Beginner:

1. Line Following Robot: A simple robot that follows a line drawn on the ground using sensors. This project helps you understand the basics of motor control, sensor integration, and microcontroller programming.
2. Sumo Robot: A small robot that can push other robots out of a circular arena. This project introduces you to robot design, motor control, and collision detection.
3. Obstacle Avoidance Robot: A robot that navigates an environment by avoiding obstacles using sensors. This project teaches you about sensor data processing, motor control, and navigation algorithms.

Intermediate:

1. Humanoid Robot: A robot that resembles a human and can perform tasks like walking, picking up objects, and waving. This project challenges you to work on complex mechanics, control systems, and AI algorithms.
2. Autonomous Lawn Mower

In [7]:
# Create the 'Model' dropdown menu with options
model_dropdown = widgets.Dropdown(
    options=['gpt-3.5-turbo', 'gpt-4-1106-preview', 'casperhansen/mixtral-instruct-awq'],
    value='gpt-3.5-turbo',
    description='Model:', 
style={'description_width': '150px'}
)

# Create the 'TVDL' dropdown menu with options
TT = widgets.Dropdown(
    options=['1', '3'],
    value='1',
    description='Test Time', 
style={'description_width': '150px'}
)

# Create the 'prompt' dropdown menu with options
prompt_dropdown = widgets.Dropdown(
    options=['None 0', 'None 1', 'None 2', 'Specific 0', 'Specific 1', 'Specific 2'],
    value='None 0',
    description='Prompt:', 
style={'description_width': '150px'}
)

# Create NumberBoxes for 'Number of Development' and 'Number of Experiments'
num_development = widgets.IntText(
    value=10,
    description='Dev. Cycle:',
style={'description_width': '150px'}
)

number_experiments = widgets.IntText(
    value=15,
    description='Experiments:',
style={'description_width': '150px'}
)


conversation_dropdown = widgets.Dropdown(
    options=['Complete', 'Exceptions+Results'],
    value='Exceptions+Results',
    description='Display Conversation:', 
    style={'description_width': '150px'}
)

run_experiments_button = widgets.Button(
    description='Run Experiment',
    button_style='info',
    tooltip='Click to run the experiment',
    icon='flask', 
    style={'description_width': '150px'}
)

# Create the dropdown widget with "Yes" and "No" options
specify_mtv = widgets.Dropdown(
    description='specify min MPa',
    options=[('No', 'no'),('Yes', 'yes'),],
    value='no',  # Default value
    disabled=False, 
    style={'description_width': '150px'}
)

# Create the integer input widget
minimal_target_value = widgets.IntText(
    value=0,
    description='min MPa Value',
    disabled=True, 
    style={'description_width': '150px'}
)

# Function to be called when dropdown value changes
def on_dropdown_change(change):
    minimal_target_value.disabled = change['new'] == 'no' # Enable the integer input


def create_experiment

# Attach the callback function to the dropdown's value changes
specify_mtv.observe(on_dropdown_change, names='value')


# Arrange the widgets in two columns
container = widgets.VBox([model_dropdown, num_development, number_experiments, conversation_dropdown, prompt_dropdown, 
                           TT, specify_mtv, minimal_target_value,run_experiments_button])



title = widgets.HTML("<h2>Settings of inverse design experiments</h2>")
settings = widgets.VBox([title, container])
display(settings)

In [14]:
# Load the text from the file
if  prompt_dropdown.value == 'None 0':
    with open('../prompts/prompts_ID_none 0.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Specific 0':
    with open('../prompts/prompts_ID_specific 0.txt', 'r') as f:
        lines = f.read().splitlines()
elif  prompt_dropdown.value == 'None 1':
    with open('../prompts/prompts_ID_none 1.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Specific 1':
    with open('../prompts/prompts_ID_specific 1.txt', 'r') as f:
        lines = f.read().splitlines()
elif  prompt_dropdown.value == 'None 2':
    with open('../prompts/prompts_ID_none 2.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Specific 2':
    with open('../prompts/prompts_ID_specific 2.txt', 'r') as f:
        lines = f.read().splitlines()

DA_role_text = lines[0]
context_text = lines[1]
iterate_text = lines[2]
VM_role_text = lines[3]

# Create the widgets with the loaded text
layout = widgets.Layout(width='auto', height='200px')  # adjust the height and width as needed
#instructions_prompt = widgets.Textarea(value=instructions_text, description='Instructions:', layout=layout)
DA_role_prompt = widgets.Textarea(value=DA_role_text, description='DA Instuctions:', layout=layout)
VM_role_prompt = widgets.Textarea(value=VM_role_text, description='VM Instuctions:', layout=layout)
context_prompt = widgets.Textarea(value=context_text, description='Design Knowledge:', layout=layout)
iterate_prompt = widgets.Textarea(value=iterate_text, description='User:', layout=layout)


container = widgets.VBox([DA_role_prompt, VM_role_prompt, context_prompt, iterate_prompt])
display(container)

# Set the 'flex' property for each widget inside main_layout
iterate_prompt.layout.flex = '2'       # Adjust the value as needed

#Instructions_box = widgets.HBox([instructions_prompt],layout=widgets.Layout(width='100%', height='100px'))
Iterate_box = widgets.HBox([iterate_prompt],layout=widgets.Layout(width='100%', height='100px'))

title = widgets.HTML("<h2>Main Prompts</h2>")
Prompts = widgets.VBox([title ,DA_role_prompt, VM_role_prompt, Iterate_box, context_prompt])

# Split the string using commas as a delimiter and convert to float numbers

budget = num_development.value
NrOfExper = number_experiments.value

desired_strength = formulation_df["Strength"].quantile(minimal_target_value.value/100)

num_entries_above_desired = (formulation_df["Strength"] >= desired_strength).sum()

# Print the result

print('SUMMARY\n' +'The design target is to achieve a strength of ',minimal_target_value.value, 'MPa within ',num_development.value,' development cycles.\n' +
      'The Experiment is repeated ',number_experiments.value,' times using the ',model_dropdown.value, ' model and the prompt strategy: ',prompt_dropdown.value,'.')
print("There are ", num_entries_above_desired,' formulations above or equal to desired_strength.')


# print(formulation_df)


     Idx_Sample  Powderkg  Liquidkg    WC  Admixturekg  Aggregateskg  \
0             0       360     162.0  0.45         14.4        1862.0   
1             1       360     180.0  0.50         14.4        1844.0   
2             2       360     198.0  0.55         14.4        1826.0   
3             3       360     216.0  0.60         14.4        1808.0   
4             4       370     166.5  0.45         14.8        1847.5   
..          ...       ...       ...   ...          ...           ...   
235         235       440     264.0  0.60         17.6        1680.0   
236         236       450     202.5  0.45         18.0        1731.5   
237         237       450     225.0  0.50         18.0        1709.0   
238         238       450     247.5  0.55         18.0        1686.5   
239         239       450     270.0  0.60         18.0        1664.0   

                                             Materials  fe3_o2  al2_o3   ca_o  \
0    Fly Ash/GGBFS-0.5/0.5, Activator: Water/NaOH/N...

/var/folders/v5/95th408n5yggqqt88z28y8z80000gn/T/ipykernel_7884/1597420129.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  formulation_df = pd.concat([formulation_df, new_row], ignore_index=True)


SUMMARY
The design target is to achieve a strength of  90 MPa within  10  development cycles.
The Experiment is repeated  15  times using the  gpt-3.5-turbo  model and the prompt strategy:  None 0 .
There are  24  formulations above or equal to desired_strength.


In [ ]:
import re
# import json

# def parse_materials(materials_str):
#     match = re.search(r'(\d+)/(\d+) FA/GGBFS', materials_str)
#     if match:
#         return int(match.group(1)) / (int(match.group(1)) + int(match.group(2)))
#     else:
#         return None
    
# def parse_curing(materials_str):
#     if "Ambient curing" in materials_str:
#         return "ambient"
#     elif "Heat curing" in materials_str:
#         return "oven"
#     else:
#         return None

# def load_data(csv_path):
#     df = pd.read_csv(csv_path)
#     df['FA_GGBFS_ratio'] = df['Materials'].apply(parse_materials)
#     df['curing'] = df['Materials'].apply(parse_curing)  # Add this line
#     return df

# df = load_data('../data/DiscoveryData_Sample.csv')

# print(df)
# # Initialize empty DataFrame
# formulation_df = pd.DataFrame(columns=["Formulation", "Strength"])

# # Loop through each row in the original data
# for idx, row in df.iterrows():
    
#     # Get necessary attributes from row
#     powder = row["Powderkg"]
#     wc = row["WC"]
#     materials = row["Materials"]

#     # Extract Fly Ash/GGBFS ratio
#     fa_ggbfs = materials.split(",")[0].split("-")[1]
    
#     # Extract curing method
#     curing_method = materials.split(",")[-1].strip()

#     # Remove unwanted string from curing method
#     curing_method = curing_method.replace(" (Rao et al. 2018)", "")
#     curing_method = curing_method.replace(" (Rao et al.)", "")
    
#     # Compressive strength
#     strength = row["fc_28dGroundTruth"]
    
#     # Create formulation string in the same format as the model's output
#     formulation = f'The formulation is Powderkg = {powder}, wc = {wc}, materials = {fa_ggbfs}, curing = {curing_method}'
    
#     # Append the formulation and its respective strength to the new DataFrame
#     new_row = pd.DataFrame({"Formulation": [formulation], "Strength": [strength]})
#     formulation_df = pd.concat([formulation_df, new_row], ignore_index=True)
